In [1]:
import numpy as np
import pandas as pd
import pyproj
import os
import math
import plotly.express as px
import geopandas as gpd
from shapely.geometry import Point
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [2]:
import data.constants as c
import data.spatial.cantons
import data.spatial.municipalities
import data.spatial.municipality_types
import data.spatial.ovgk
import data.spatial.utils
import data.spatial.zones
import data.utils
import data.utils


### Prepare Data

#### Define Functions

In [3]:
import os
import platform

def get_data_folder_path():
    # Get the current operating system
    os_type = platform.system()
    user_name = os.getlogin()

    # Define data folder paths for different systems
    if os_type == 'Windows' and user_name == 'Alice':
        data_folder_path = f"C:\\Users\\{user_name}\\Documents\\Data"
    elif os_type == 'Linux' and user_name == 'salathem':
        data_folder_path = '/cluster/home/salathem/Programming/data/'
    elif os_type == 'Darwin':
        data_folder_path = '/Users/Marco/Library/CloudStorage/OneDrive-Persönlich/ETHZ/Agent Based Modeling/data/'
    else:
        raise Exception("Unsupported system configuration")

    return data_folder_path

In [4]:
def configure(context):
    context.config("data_path")
    context.stage("data.spatial.municipalities")
    context.stage("data.spatial.zones")
    context.stage("data.spatial.municipality_types")
    context.stage("data.statpop.density")
    context.stage("data.spatial.ovgk")

In [5]:
def fix_marital_status(df):
    """ Makes young people, who are separated, be treated as single! """
    df.loc[
        (df["marital_status"] == c.MARITAL_STATUS_SEPARATE) & (df["age"] < c.SEPARATE_SINGLE_THRESHOLD)
        , "marital_status"] = c.MARITAL_STATUS_SINGLE
    df.loc[:, "marital_status"] = df.loc[:, "marital_status"].astype(int)

In [6]:
def execute_person(path):
    data_path = path

    df_mz_persons = pd.read_csv(
        "%s/microzensus/zielpersonen.csv" % data_path,
        sep = ",", encoding = "latin1", parse_dates = ["USTag"]
    )

    df_mz_persons["age"] = df_mz_persons["alter"]
    df_mz_persons["sex"] = df_mz_persons["gesl"] - 1 # Make zero-based
    df_mz_persons["person_id"] = df_mz_persons["HHNR"]
    df_mz_persons["person_weight"] = df_mz_persons["WP"]
    df_mz_persons["date"] = df_mz_persons["USTag"]

    # Marital status
    df_mz_persons.loc[df_mz_persons["zivil"] == 1, "marital_status"] = c.MARITAL_STATUS_SINGLE
    df_mz_persons.loc[df_mz_persons["zivil"] == 2, "marital_status"] = c.MARITAL_STATUS_MARRIED
    df_mz_persons.loc[df_mz_persons["zivil"] == 3, "marital_status"] = c.MARITAL_STATUS_SEPARATE
    df_mz_persons.loc[df_mz_persons["zivil"] == 4, "marital_status"] = c.MARITAL_STATUS_SEPARATE
    df_mz_persons.loc[df_mz_persons["zivil"] == 5, "marital_status"] = c.MARITAL_STATUS_SINGLE
    df_mz_persons.loc[df_mz_persons["zivil"] == 6, "marital_status"] = c.MARITAL_STATUS_MARRIED
    df_mz_persons.loc[df_mz_persons["zivil"] == 7, "marital_status"] = c.MARITAL_STATUS_SEPARATE

    # Driving license
    df_mz_persons["driving_license"] = df_mz_persons["f20400a"] == 1

    # Car availability
    df_mz_persons["car_availability"] = c.CAR_AVAILABILITY_NEVER
    df_mz_persons.loc[df_mz_persons["f42100e"] == 1, "car_availability"] = c.CAR_AVAILABILITY_ALWAYS
    df_mz_persons.loc[df_mz_persons["f42100e"] == 2, "car_availability"] = c.CAR_AVAILABILITY_SOMETIMES
    df_mz_persons.loc[df_mz_persons["f42100e"] == 3, "car_availability"] = c.CAR_AVAILABILITY_NEVER

    # Employment (TODO: I know that LIMA uses a more fine-grained category here)
    df_mz_persons["employed"] = df_mz_persons["f40800_01"] != -99

    # Infer age class
    df_mz_persons["age_class"] = np.digitize(df_mz_persons["age"], c.AGE_CLASS_UPPER_BOUNDS)

    # Fix marital status
    fix_marital_status(df_mz_persons)

    # Day of the observation
    df_mz_persons["weekend"] = False
    df_mz_persons.loc[df_mz_persons["tag"] == 6, "weekend"] = True
    df_mz_persons.loc[df_mz_persons["tag"] == 7, "weekend"] = True

    # Here we extract a bit more than Kirill, but most likely it will be useful later

    df_mz_persons["subscriptions_ga"] = df_mz_persons["f41610a"] == 1
    df_mz_persons["subscriptions_halbtax"] = df_mz_persons["f41610b"] == 1
    df_mz_persons["subscriptions_verbund"] = df_mz_persons["f41610c"] == 1
    df_mz_persons["subscriptions_strecke"] = df_mz_persons["f41610d"] == 1
    df_mz_persons["subscriptions_gleis7"] = df_mz_persons["f41610e"] == 1
    df_mz_persons["subscriptions_junior"] = df_mz_persons["f41610f"] == 1
    df_mz_persons["subscriptions_other"] = df_mz_persons["f41610g"] == 1

    df_mz_persons["subscriptions_ga_class"] = df_mz_persons["f41651"] == 1
    df_mz_persons["subscriptions_verbund_class"] = df_mz_persons["f41653"] == 1
    df_mz_persons["subscriptions_strecke_class"] = df_mz_persons["f41654"] == 1

    # Education
    df_mz_persons["highest_education"] = np.nan
    df_mz_persons.loc[df_mz_persons["HAUSB"].isin([1, 2, 3, 4]), "highest_education"] = "primary"
    df_mz_persons.loc[df_mz_persons["HAUSB"].isin([5, 6, 7, 8, 9, 10, 11, 12]), "highest_education"] = "secondary"
    df_mz_persons.loc[df_mz_persons["HAUSB"].isin([13, 14, 15, 16]), "highest_education"] = "tertiary_professional"
    df_mz_persons.loc[df_mz_persons["HAUSB"].isin([17, 18, 19]), "highest_education"] = "tertiary_academic"
    df_mz_persons["highest_education"] = df_mz_persons["highest_education"].astype("category")

    # Parking
    df_mz_persons["parking_work"] = "unknown"
    df_mz_persons.loc[df_mz_persons["f41300"] == 1, "parking_work"] = "free"
    df_mz_persons.loc[df_mz_persons["f41300"] == 2, "parking_work"] = "paid"
    df_mz_persons.loc[df_mz_persons["f41300"] == 3, "parking_work"] = "no"
    df_mz_persons["parking_work"] = df_mz_persons["parking_work"].astype("category")

    df_mz_persons["parking_education"] = "unknown"
    df_mz_persons.loc[df_mz_persons["f41301"] == 1, "parking_education"] = "free"
    df_mz_persons.loc[df_mz_persons["f41301"] == 2, "parking_education"] = "paid"
    df_mz_persons.loc[df_mz_persons["f41301"] == 3, "parking_education"] = "no"
    df_mz_persons["parking_education"] = df_mz_persons["parking_education"].astype("category")

    df_mz_persons["parking_cost_work"] = np.maximum(0, df_mz_persons["f41400"].astype(float))
    df_mz_persons["parking_cost_education"] = np.maximum(0, df_mz_persons["f41401"].astype(float))

    # Wrap up
    df_mz_persons = df_mz_persons[[
        "person_id",
        "age", "sex",
        "marital_status",
        "driving_license",
        "car_availability",
        "employed",
        "highest_education",
        "parking_work", "parking_cost_work",
        "parking_education", "parking_cost_education",
        "subscriptions_ga",
        "subscriptions_halbtax",
        "subscriptions_verbund",
        "subscriptions_strecke",
        "subscriptions_gleis7",
        "subscriptions_junior",
        "subscriptions_other",
        "subscriptions_ga_class",
        "subscriptions_verbund_class",
        "subscriptions_strecke_class",
        "age_class", "person_weight",
        "weekend", "date"
    ]]

    # # Merge in the other data sets
    # df_mz_households = context.stage("data.microcensus.households")
    # df_mz_trips = context.stage("data.microzensus.trips")

    # df_mz_persons = pd.merge(df_mz_persons, df_mz_households)
    # df_mz_persons = data.microzensus.income.impute(df_mz_persons)

    # remove_ids = set(df_mz_persons["person_id"]) - set(df_mz_trips["person_id"])
    # initial_size = len(df_mz_persons)

    # df_mz_persons = df_mz_persons[~df_mz_persons["person_id"].isin(remove_ids)]

    # # Note: Around 7000 of them are those, which do not even have an activity chain in the first place
    # # because they have not been asked.
    # print("  Removed %d (%.2f%%) persons from MZ because of insufficient trip data" % (
    #     len(remove_ids), 100.0 * len(remove_ids) / initial_size
    # ))

    # # Add car passenger flag
    # car_passenger_ids = df_mz_trips.loc[df_mz_trips["mode"] == "car_passenger", "person_id"].unique()
    # df_mz_persons["is_car_passenger"] = df_mz_persons["person_id"].isin(car_passenger_ids)

    return df_mz_persons

In [7]:
def execute_household(path):
    data_path = path

    df_mz_households = pd.read_csv(
        "%s/microzensus/haushalte.csv" % data_path, sep=",", encoding="latin1")

    # Simple attributes
    df_mz_households["home_structure"] = df_mz_households["W_STRUKTUR_AGG_2000"]
    df_mz_households["household_size"] = df_mz_households["hhgr"]
    df_mz_households["number_of_cars"] = np.maximum(0, df_mz_households["f30100"])
    df_mz_households["number_of_bikes"] = df_mz_households["f32200a"]
    df_mz_households["person_id"] = df_mz_households["HHNR"]
    df_mz_households["household_weight"] = df_mz_households["WM"]

    # Income
    df_mz_households["income_class"] = df_mz_households["F20601"] - 1  # Turn into zero-based class
    df_mz_households["income_class"] = np.maximum(-1, df_mz_households["income_class"])  # Make all "invalid" entries -1

    # Convert coordinates to LV95
    coords = df_mz_households[["W_X_CH1903", "W_Y_CH1903"]].values
    transformer = pyproj.Transformer.from_crs(c.CH1903, c.CH1903_PLUS)
    x, y = transformer.transform(coords[:, 0], coords[:, 1])
    df_mz_households.loc[:, "home_x"] = x
    df_mz_households.loc[:, "home_y"] = y

    # Class variable for number of cars
    df_mz_households["number_of_cars_class"] = 0
    df_mz_households.loc[df_mz_households["number_of_cars"] > 0, "number_of_cars_class"] = np.minimum(
        c.MAX_NUMBER_OF_CARS_CLASS, df_mz_households["number_of_cars"])

    # Bike availability depends on household size. (TODO: Would it make sense to use the same concept for cars?)
    df_mz_households["number_of_bikes_class"] = c.BIKE_AVAILABILITY_FOR_NONE
    df_mz_households.loc[
        df_mz_households["number_of_bikes"] > 0, "number_of_bikes_class"] = c.BIKE_AVAILABILITY_FOR_SOME
    df_mz_households.loc[
        df_mz_households["number_of_bikes"] >= df_mz_households["household_size"],
        "number_of_bikes_class"] = c.BIKE_AVAILABILITY_FOR_ALL

    # Household size class
    data.utils.assign_household_class(df_mz_households)

    # Region information
    # (acc. to Analyse der SP-Befragung 2015 zur Verkehrsmodus- und Routenwahl)
    df_mz_households["canton_id"] = df_mz_households["W_KANTON"]
    df_mz_households = data.spatial.cantons.impute_sp_region(df_mz_households)

    # # Impute spatial information
    # df_municipalities = context.stage("data.spatial.municipalities")[0]
    # df_zones = context.stage("data.spatial.zones")
    # df_municipality_types = context.stage("data.spatial.municipality_types")

    # df_spatial = pd.DataFrame(df_mz_households[["person_id", "home_x", "home_y"]])
    # df_spatial = data.spatial.utils.to_gpd(context, df_spatial, "home_x", "home_y")
    # df_spatial = data.spatial.utils.impute(context, df_spatial, df_municipalities, "person_id", "municipality_id")
    # df_spatial = data.spatial.zones.impute(df_spatial, df_zones)
    # df_spatial = data.spatial.municipality_types.impute(df_spatial, df_municipality_types)

    # df_mz_households = pd.merge(
    #     df_mz_households, df_spatial[["person_id", "zone_id", "municipality_type"]],
    #     on="person_id"
    # )

    # df_mz_households["home_zone_id"] = df_mz_households["zone_id"]

    # # Impute density
    # data.statpop.density.impute(context.stage("data.statpop.density"), df_mz_households, "home_x", "home_y")

    # # Impute OV Guteklasse
    # print("Imputing ÖV Güteklasse ...")
    # df_ovgk = context.stage("data.spatial.ovgk")
    # df_spatial = data.spatial.ovgk.impute(context, df_ovgk, df_spatial, ["person_id"])
    # df_mz_households = pd.merge(df_mz_households, df_spatial[["person_id", "ovgk"]], on=["person_id"], how="left")

    # Wrap it up
    return df_mz_households[[
        "person_id", "household_size", "number_of_cars", "number_of_bikes", "income_class",
        "home_x", "home_y", "household_size_class", "number_of_cars_class", "number_of_bikes_class", "household_weight",
        "sp_region", "canton_id"]]


#### Run Preparation and Merge

In [8]:
data_folder_path = get_data_folder_path()
df_mz_persons = execute_person(data_folder_path)
df_mz_households = execute_household(data_folder_path)

In [9]:
df = pd.merge(df_mz_persons, df_mz_households, on='person_id', how='left')


In [10]:
df

,person_id,age,sex,marital_status,driving_license,car_availability,employed,highest_education,parking_work,parking_cost_work,...,number_of_bikes,income_class,home_x,home_y,household_size_class,number_of_cars_class,number_of_bikes_class,household_weight,sp_region,canton_id
0,100004,50,1,1.0,True,0,True,secondary,unknown,0.0,...,3,3,2.655084e+06,1.255417e+06,2,1,2,0.781952,3,19
1,100010,8,1,0.0,False,2,False,NaN,unknown,0.0,...,2,-1,2.724147e+06,1.119942e+06,1,1,2,0.394216,2,21
2,100021,68,1,1.0,True,1,False,primary,unknown,0.0,...,2,1,2.758533e+06,1.249882e+06,1,1,2,0.548396,3,17
3,100028,61,1,1.0,True,0,True,tertiary_academic,unknown,0.0,...,4,3,2.547342e+06,1.178977e+06,1,2,2,0.930521,2,22
4,100037,30,1,1.0,False,2,False,primary,unknown,0.0,...,0,2,2.501773e+06,1.116555e+06,3,1,0,1.337850,1,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57085,499969,15,0,0.0,False,2,False,primary,unknown,0.0,...,2,-1,2.499968e+06,1.116573e+06,4,1,1,0.162240,1,25
57086,499994,10,0,0.0,False,2,False,NaN,unknown,0.0,...,5,-1,2.666612e+06,1.225863e+06,4,1,2,0.185661,2,3
57087,499995,48,0,2.0,True,0,True,tertiary_academic,unknown,0.0,...,0,8,2.498129e+06,1.114995e+06,1,1,0,0.616858,1,25
57088,499996,68,1,2.0,True,0,False,tertiary_academic,unknown,0.0,...,0,2,2.685662e+06,1.248720e+06,0,1,0,2.901484,1,1


#### Filter Data for Scenario

In [11]:
# Load geographic data from a shapefile
shapefile_path = data_folder_path + '/scenarios/Zurich/ScenarioBoundary/zurich_city_5km.shp'  # please replace with your shapefile path
gdf = gpd.read_file(shapefile_path)

# Ensure the GeoDataFrame is in the CH1903 coordinate system
# If the gdf is not in CH1903, you would convert it like this:
# gdf = gdf.to_crs(epsg=21781)  # EPSG 21781 is the code for CH1903/LV03

# Get the polygon for Zurich city
zurich_polygon = gdf[gdf['scenario'] == 'zurich_city'].iloc[0]['geometry']

# Create Point geometries for home locations
df['home_point'] = df.apply(lambda row: Point(row['home_x'], row['home_y']), axis=1)

# Convert the DataFrame to a GeoDataFrame
gdf_points = gpd.GeoDataFrame(df, geometry='home_point', crs=gdf.crs)

# Filter trips where home points are within the Zurich city polygon
df = gdf_points[gdf_points['home_point'].within(zurich_polygon)]
df = pd.DataFrame(df.drop(columns='home_point'))


In [12]:
df.to_csv(data_folder_path + '/microzensus/population.csv')

### Plot Data

In [13]:
variables_hh = [
    "person_id", "household_size", "number_of_cars", "number_of_bikes", "income_class",
    "home_x", "home_y", "household_size_class", "number_of_cars_class", "number_of_bikes_class", "household_weight",
    "sp_region", "canton_id"]

variables_p = [
    "age", "sex",
    "marital_status",
    "driving_license",
    "car_availability",
    "employed",
    "highest_education",
    "parking_work", "parking_cost_work",
    "parking_education", "parking_cost_education",
    "subscriptions_ga",
    "subscriptions_halbtax",
    "subscriptions_verbund",
    "subscriptions_strecke",
    "subscriptions_gleis7",
    "subscriptions_junior",
    "subscriptions_other",
    "subscriptions_ga_class",
    "subscriptions_verbund_class",
    "subscriptions_strecke_class",
    "age_class", "person_weight",
    "weekend", "date"
]

# Assuming df is your DataFrame and variables is your list of variables to plot
variables = variables_hh + variables_p  # Combine your two lists of variables

# Calculate the number of rows needed for the subplots (3 columns)
num_rows = math.ceil(len(variables) / 3)

# Create a subplot figure with the calculated number of rows and 3 columns
fig = make_subplots(rows=num_rows, cols=3, subplot_titles=variables)

# Add a histogram to each subplot for the respective variable
for i, var in enumerate(variables):
    row = math.ceil((i + 1) / 3)
    col = (i % 3) + 1
    fig.add_trace(
        go.Histogram(x=df[var], name=var),
        row=row, col=col
    )

# Update layout if needed
fig.update_layout(
    title_text='Distributions of Variables',
    height=300 * num_rows,  # Adjust the height based on the number of rows
)

# Show the figure
fig.show()